# Scraper Workbook

## Import libraries and modules

In [1]:
import pandas as pd
import json
import math
import random
import re
import string
from typing import List, TypedDict
from urllib.parse import urljoin

from loguru import logger as log
from scrapfly import ScrapeApiResponse, ScrapeConfig, ScrapflyClient

import psycopg2
from psycopg2 import extras
from elasticsearch import Elasticsearch
import hashlib
import hidden

import warnings
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

## Set up database connections

In [2]:
# Set up Elasticsearch connection
secrets = hidden.elastic()

es = Elasticsearch(
    f"https://{secrets['host']}:{secrets['port']}",
    ca_certs=secrets['ca'],
    basic_auth=(secrets['user'], secrets['pass']),
    verify_certs=False
)

/opt/miniconda3/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://es.matrix4p.com:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [3]:
# Set up PostgreSQL connection
sql_string = hidden.psycopg2(hidden.postgres())
conn = psycopg2.connect(sql_string, connect_timeout=3)

## Define scraper functions

In [4]:
def search_hotel(query: str, session: ScrapflyClient):
    """
    search for hotel page from given query.
    """
    log.info(f"searching: {query}")
    url = "https://www.tripadvisor.com/data/graphql/ids"
    
    payload = json.dumps(
        [
            {
                "query": "c9d791589f937ec371723f236edc7c6b",
                "variables": {
                    "request": {
                        "query": query,
                        "limit": 10,
                        "scope": "WORLDWIDE",
                        "locale": "en-US",
                        "scopeGeoId": 1,
                        "searchCenter": None,
                        "types": [
                            "LOCATION",
                            "QUERY_SUGGESTION",
                            "USER_PROFILE",
                            "RESCUE_RESULT"
                        ],
                        "locationTypes": [
                            #   "GEO",
                            #   "AIRPORT",
                            "ACCOMMODATION",
                            #   "ATTRACTION",
                            #   "ATTRACTION_PRODUCT",
                            #   "EATERY",
                            #   "NEIGHBORHOOD",
                            #   "AIRLINE",
                            #   "SHOPPING",
                            #   "UNIVERSITY",
                            #   "GENERAL_HOSPITAL",
                            #   "PORT",
                            #   "FERRY",
                            #   "CORPORATION",
                            #   "VACATION_RENTAL",
                            #   "SHIP",
                            #   "CRUISE_LINE",
                            #   "CAR_RENTAL_OFFICE"
                        ],
                        "userId": None,
                        "context": {
                            "searchSessionId": "E79EC7492FFD0BE3C904AA89F02DC1AA1665037199910ssid",
                            "typeaheadId": "1665037209369",
                            "uiOrigin": "SINGLE_SEARCH_HERO",
                            "routeUid": "LIT@eBE@IIdXLLmlCycNjnyj"
                        },
                        "articleCategories": [
                            "default",
                            "love_your_local",
                            "insurance_lander"
                        ],
                        "enabledFeatures": [
                            "typeahead-q"
                        ]
                    }
                }
            }
        ]
    )

    headers = {
        # we need to generate a random request ID for this request to succeed
        "content-type": "application/json",
        "x-requested-by": "".join(random.choice(string.ascii_lowercase + string.digits) for i in range(64)),
    }
    result = session.scrape(
        ScrapeConfig(
            url=url,
            country="US",
            headers=headers,
            body=payload,
            method="POST",
            asp=True,
        )
    )
    data = json.loads(result.content)

    # return the most matched result
    log.info(f'found {len(data[0]["data"]["Typeahead_autocomplete"]["results"])} results, taking the most matched result')
    return data[0]["data"]["Typeahead_autocomplete"]["results"][0]["details"]
    
    # for res in data[0]['data']['Typeahead_autocomplete']['results']:
    #     # confirming if the `localizedName` of `res` matches the query, and has `latitude` value
    #     if (query.split(',')[0].lower() in res['details']['localizedName'].lower()) and (res['details']['latitude'] != None):
    #         matched_result = res
    # return matched_result["details"]


def extract_page_manifest(html):
    """extract javascript state data hidden in TripAdvisor HTML pages"""
    data = re.findall(r"pageManifest:({.+?})};", html, re.DOTALL)[0]
    return json.loads(data)


def extract_named_urql_cache(urql_cache: dict, pattern: str):
    """extract named urql response cache from hidden javascript state data"""
    data = json.loads(next(v["data"] for k, v in urql_cache.items() if pattern in v["data"]))
    return data


class Review(TypedDict):
    id: str
    date: str
    rating: str
    title: str
    text: str
    votes: int
    url: str
    language: str
    platform: str
    author_id: str
    author_name: str
    author_username: str


def parse_reviews(result: ScrapeApiResponse) -> List[Review]:
    """Parse reviews from a review page"""
    page_data = extract_page_manifest(result.content)
    review_cache = extract_named_urql_cache(page_data["urqlCache"], '"reviewListPage"')
    parsed = []
    # review data contains loads of information, let's parse only the basic in this tutorial
    for review in review_cache["locations"][0]["reviewListPage"]["reviews"]:
        parsed.append(
            {
                "id": review["id"],
                "date": review["publishedDate"],
                "rating": review["rating"],
                "title": review["title"],
                "text": review["text"],
                "votes": review["helpfulVotes"],
                "url": review["route"]["url"],
                "language": review["language"],
                "platform": review["publishPlatform"],
                "author_id": review["userProfile"]["id"],
                "author_name": review["userProfile"]["displayName"],
                "author_username": review["userProfile"]["username"],
            }
        )
    return parsed


class Hotel(TypedDict):
    name: str
    id: int
    type: str
    description: str
    rating: int
    rating_count: int
    features: List[str]
    stars: str or None


def parse_hotel_info(data: dict) -> Hotel:
    """parse hotel data from TripAdvisor javascript state to something more readable"""
    parsed = {}
    # there's a lot of information in hotel data, in this tutorial let's extract the basics:
    parsed["name"] = data["name"]
    parsed["id"] = data["locationId"]
    parsed["type"] = data["accommodationType"]
    parsed["description"] = data["locationDescription"]
    parsed["rating"] = data["reviewSummary"]["rating"]
    parsed["rating_count"] = data["reviewSummary"]["count"]
    # for hotel "features" lets just extract the names:
    parsed["features"] = []
    for amenity_type, values in data["detail"]["hotelAmenities"]["highlightedAmenities"].items():
        for value in values:
            parsed["features"].append(f"{amenity_type}_{value['amenityNameLocalized'].lower()}")

    if star_rating := data["detail"]["starRating"]:
        parsed["stars"] = star_rating[0]["tagNameLocalized"]
    return parsed  # type: ignore


class HotelAllData(TypedDict):
    hotel: str
    url: str
    info: Hotel
    reviews: List[Review]
    price: List[dict]
    page: dict


def scrape_hotel(hotel: str, url: str, session: ScrapflyClient) -> HotelAllData:
    """Scrape all hotel data: information, pricing and reviews"""
    log.info(f"scraping: {url}")
    first_page = session.scrape(ScrapeConfig(url=url, country="US"))
    if first_page.prevent_extra_usage():
        print(f'{first_page.remaining_quota} left.')
        print(f'{first_page.cost} USD per extra API call.')
        return first_page  # type: ignore
    page_data = extract_page_manifest(first_page.content)
    #return page_data

    # price data keys are dynamic first we need to find the full key name
    _pricing_key = next(
        (key for key, val in page_data['urqlCache'].items() if 'HPS_getWebHROffers' in val['data'])
    )
    pricing_details = list()
    price_page = json.loads(page_data['urqlCache'][_pricing_key]['data'])['HPS_getWebHROffers']
    for offer_key in ['chevronOffers', 'textLinkOffers']:
        if any([source['status'] == 'AVAILABLE' for source in price_page[offer_key]]):
            for source in price_page[offer_key]:
                if source['status'] == 'AVAILABLE':
                    pricing = dict()
                    pricing['vendorName'] = source['data']['dataAtts']['vendorName']
                    pricing['perNight'] = source['data']['dataAtts']['perNight']
                    pricing_details.append(pricing)
    log.info(f"found {len(pricing_details)} prices from {[pricing['vendorName'] for pricing in pricing_details]}")

    # We can extract data from Graphql cache embeded in the page
    # TripAdvisor is using: https://github.com/FormidableLabs/urql as their graphql client
    try:
        hotel_cache = extract_named_urql_cache(page_data["urqlCache"], '"locationDescription"')
        hotel_info = hotel_cache["locations"][0]
        info = parse_hotel_info(hotel_info)
    except:
        hotel_cache = json.loads(next(v["data"] for k, v in page_data["urqlCache"].items() 
                                    if ('"locationDescription"' in v["data"]) and 
                                       ('"accommodationType"' in v["data"]) and 
                                       ('"name"' in v["data"])))
        hotel_info = hotel_cache["locations"][0]
        info = parse_hotel_info(hotel_info)

    # for reviews we first need to scrape multiple pages
    # so, first let's find total amount of pages
    total_reviews = hotel_info["reviewSummary"]["count"]
    _review_page_size = 10
    total_review_pages = int(math.ceil(total_reviews / _review_page_size))
    log.info(f"found {total_reviews} reviews in {total_review_pages} pages, taking first one")
    # then we can scrape all review pages one by one
    # note: in review url "or" stands for "offset reviews"
    review_urls = [
        url.replace("-Reviews-", f"-Reviews-or{_review_page_size * i}-") for i in range(2, total_review_pages + 1)
    ]
    assert len(set(review_urls)) == len(review_urls)
    reviews = parse_reviews(first_page)
    n = 0
    for url in review_urls:
        if n > 80: break
        result = session.scrape(ScrapeConfig(url=url, country="US"))
        try:
            reviews.extend(parse_reviews(result))
        except:
            continue
        n += 1
    log.info(f"{len(reviews)} reviews scraped")

    return {
        "hotel": hotel,
        "url": url,
        "price": pricing_details,
        "info": info,
        "reviews": reviews,
        "page": page_data
    }


def es_insert(result_hotel):
    # Generate a sha256 value from `url` for Elasticsearch doc id
    m = hashlib.sha256()
    m.update(result_hotel['url'].encode())
    pkey = m.hexdigest()
    
    for key, val in result_hotel.items():
        # Add/refresh `val` to Elasticsearch with index `key`
        if key == 'url': continue
        elif key == 'hotel':
            doc = {key: val, 'url': result_hotel['url']}
            res = es.index(index=key, id=pkey, document=doc)
        elif key == 'price':
            res = es.index(index=key, id=pkey, document={"group": "price", "price": val})
        elif key == 'reviews':
            res = es.index(index=key, id=pkey, document={"group": "reviews", "reviews": val})
        elif key == 'page':
            res = es.index(index=key, id=pkey, document={"group": "page",
                                                         "page": val['urqlCache']})
                                                         #"page": val})
        else:
            res = es.index(index=key, id=pkey, document=val)
        print(f"Added document as index '{key}'")
        print(res['result'])


def pg_insert(conn, result_hotel):
    # Insert the `url` and `result_hotel` into PostgreSQL
    with conn.cursor() as cur:
        sql = """INSERT INTO hotels (hotel, url, info, price, reviews, page) 
                   VALUES (%s, %s, %s, %s, %s, %s);"""
        cur.execute(sql, (result_hotel['hotel'],
                          result_hotel['url'],
                          extras.Json(result_hotel['info']),
                          extras.Json(result_hotel['price']),
                          extras.Json(result_hotel['reviews']),
                          extras.Json(result_hotel['page'])))
    conn.commit()


def run(hotel, query):
    secrets_scrapfly = hidden.scrapfly()
    key = secrets_scrapfly['key']
    with ScrapflyClient(key=key, max_concurrency=20) as session:
        hotel_url = "https://www.tripadvisor.com/" + (search_hotel(query, session))["url"]
        result_hotel = scrape_hotel(
            hotel,
            hotel_url,
            session,
        )

        return result_hotel

## Scrape missing hotel in `ta_hotels` from TripAdvisor

In [8]:
# Load the Travel + Leisure World's Best Hotels 2022 dataset
hotels_2022_df = pd.read_csv('./data/100_hotels_2022.csv', encoding='latin1')

# Load scraped TripAdvisor hotels info dataset
ta_hotels_df = pd.read_pickle('./data/ta_hotels.pickle.zip')

for hotel in hotels_2022_df.Hotel:
    #if not any(hotel.lower() in str.lower(row['name']) for row in ta_hotels_df['info']):  # check if hotel already scraped
    if not hotel in ta_hotels_df.hotel.tolist():  # check if hotel already scraped
        query = hotel \
            + ', ' + hotels_2022_df.loc[hotels_2022_df.Hotel == hotel, 'Location'].values[0] \
            + ', ' + hotels_2022_df.loc[hotels_2022_df.Hotel == hotel, 'Country'].values[0]
        result_hotel = run(hotel, query)

        # Insert result to Elasticsearch
        try:
            es_insert(result_hotel=result_hotel)
        except:
            try: 
                es_insert(result_hotel=result_hotel)
            except:
                pass

        # Insert result to PostgreSQL database
        pg_insert(conn=conn, result_hotel=result_hotel)

        # Insert result to dataframe and save to pickle file
        row = pd.Series(result_hotel).to_frame().T
        ta_hotels_df = pd.concat([ta_hotels_df, row], ignore_index=True)
        ta_hotels_df.to_pickle('./data/ta_hotels.pickle.zip')
        
        #break
    else: print('All hotels are scraped and stored')

2022-10-07 18:59:53.972 | INFO     | __main__:search_hotel:5 - searching: Wentworth Mansion, Charleston, United States
2022-10-07 18:59:56.432 | INFO     | __main__:search_hotel:85 - found 2 results, taking the most matched result
2022-10-07 18:59:56.434 | INFO     | __main__:scrape_hotel:191 - scraping: https://www.tripadvisor.com//Hotel_Review-g54171-d111475-Reviews-Wentworth_Mansion-Charleston_South_Carolina.html
2022-10-07 18:59:59.520 | INFO     | __main__:scrape_hotel:210 - found 3 prices from ['Expedia.com', 'Hotels.com', 'Official Site']
2022-10-07 18:59:59.521 | INFO     | __main__:scrape_hotel:229 - found 1212 reviews in 122 pages, taking first one
2022-10-07 19:02:34.371 | INFO     | __main__:scrape_hotel:246 - 820 reviews scraped


Added document as index 'hotel'
created
Added document as index 'price'
created
Added document as index 'info'
created
Added document as index 'reviews'
created


2022-10-07 19:02:44.973 | INFO     | __main__:search_hotel:5 - searching: Taj Lands End, Mumbai, India
2022-10-07 19:02:46.285 | INFO     | __main__:search_hotel:85 - found 6 results, taking the most matched result
2022-10-07 19:02:46.286 | INFO     | __main__:scrape_hotel:191 - scraping: https://www.tripadvisor.com//Hotel_Review-g304554-d304604-Reviews-Taj_Lands_End_Mumbai-Mumbai_Maharashtra.html
2022-10-07 19:02:48.938 | INFO     | __main__:scrape_hotel:210 - found 3 prices from ['Official Site', 'Expedia.com', 'ZenHotels.com']
2022-10-07 19:02:48.940 | INFO     | __main__:scrape_hotel:229 - found 5916 reviews in 592 pages, taking first one
2022-10-07 19:05:42.148 | INFO     | __main__:scrape_hotel:246 - 820 reviews scraped


Added document as index 'hotel'
created
Added document as index 'price'
created
Added document as index 'info'
created
Added document as index 'reviews'
created
Added document as index 'hotel'
updated
Added document as index 'price'
updated
Added document as index 'info'
updated
Added document as index 'reviews'
updated


2022-10-07 19:05:42.840 | INFO     | __main__:search_hotel:5 - searching: Birkenhead House, Hermanus, South Africa
2022-10-07 19:05:44.791 | INFO     | __main__:search_hotel:85 - found 2 results, taking the most matched result
2022-10-07 19:05:44.792 | INFO     | __main__:scrape_hotel:191 - scraping: https://www.tripadvisor.com//Hotel_Review-g312663-d459781-Reviews-Birkenhead_House-Hermanus_Overstrand_Overberg_District_Western_Cape.html
2022-10-07 19:05:46.984 | INFO     | __main__:scrape_hotel:210 - found 6 prices from ['Hotels.com', 'Expedia.com', 'Travelocity', 'Priceline', 'Orbitz.com', 'eDreams']
2022-10-07 19:05:46.985 | INFO     | __main__:scrape_hotel:229 - found 528 reviews in 53 pages, taking first one
2022-10-07 19:07:31.373 | INFO     | __main__:scrape_hotel:246 - 488 reviews scraped


Added document as index 'hotel'
created
Added document as index 'price'
created
Added document as index 'info'
created
Added document as index 'reviews'
created
Added document as index 'hotel'
updated
Added document as index 'price'
updated
Added document as index 'info'
updated
Added document as index 'reviews'
updated


## Scrape one hotel link and insert to databases

In [ ]:
# Get the hotel to scrape
hotels_2022_df.iloc[60]

### Scrape the link of the hotel

In [ ]:
url = 'https://www.tripadvisor.com//Hotel_Review-g187147-d497267-Reviews-or550-La_Reserve_Paris_Hotel_and_Spa-Paris_Ile_de_France.html'

secrets_scrapfly = hidden.scrapfly()
key = secrets_scrapfly['key']
with ScrapflyClient(key=key, max_concurrency=20) as session:
    hotel_url = url
    result_hotel = scrape_hotel(
        hotel,
        hotel_url,
        session,
    )

### Insert the result to Elasticsearch

In [3]:
# Set up Elasticsearch connection
secrets = hidden.elastic()

es = Elasticsearch(
    f"https://{secrets['host']}:{secrets['port']}",
    ca_certs=secrets['ca'],
    basic_auth=(secrets['user'], secrets['pass']),
    verify_certs=False
)

# Insert result to Elasticsearch
try:  # try/except in case the connection to Elasticsearch got lost
    es_insert(result_hotel=result_hotel)
except:
    try: # try/except to pass th mapping error
        es_insert(result_hotel=result_hotel)
    except:
        pass

### Insert the result to PostgreSQL and save to Pickle file

In [7]:
# Set up PostgreSQL connection
sql_string = hidden.psycopg2(hidden.postgres())
conn = psycopg2.connect(sql_string, connect_timeout=3)

# Insert result to PostgreSQL database
pg_insert(conn=conn, result_hotel=result_hotel)

# Insert result to df
row = pd.Series(result_hotel).to_frame().T
ta_hotels_df = pd.concat([ta_hotels_df, row], ignore_index=True)
ta_hotels_df.to_pickle('./data/ta_hotels.pickle.zip')

## Retrieve one hotel pricing and insert to databases

### Scrape the link of the hotel and retrieve prices

In [150]:
# Load scraped TripAdvisor hotels info dataset
ta_hotels_df = pd.read_pickle('./data/ta_hotels.pickle.zip')

hotel = 'The Lowell'

url = ta_hotels_df.loc[ta_hotels_df.hotel == hotel, 'url'].values[0]
print(url)

https://www.tripadvisor.com//Hotel_Review-g60763-d93585-Reviews-or480-Lowell_Hotel-New_York_City_New_York.html


In [151]:
secrets_scrapfly = hidden.scrapfly()
key = secrets_scrapfly['key']
with ScrapflyClient(key=key, max_concurrency=20) as session:
    """Scrape hotel pricing"""
    log.info(f"scraping: {url}")
    first_page = session.scrape(ScrapeConfig(url=url, country="US"))

page_data = extract_page_manifest(first_page.content)

# price data keys are dynamic first we need to find the full key name
_pricing_key = next(
    (key for key, val in page_data['urqlCache'].items() if 'HPS_getWebHROffers' in val['data'])
)
pricing_details = list()
price_page = json.loads(page_data['urqlCache'][_pricing_key]['data'])['HPS_getWebHROffers']
for offer_key in ['chevronOffers', 'textLinkOffers']:
    if any([source['status'] == 'AVAILABLE' for source in price_page[offer_key]]):
        for source in price_page[offer_key]:
            if source['status'] == 'AVAILABLE':
                pricing = dict()
                pricing['vendorName'] = source['data']['dataAtts']['vendorName']
                pricing['perNight'] = source['data']['dataAtts']['perNight']
                pricing_details.append(pricing)

log.info(f"found {len(pricing_details)} prices from {[pricing['vendorName'] for pricing in pricing_details]}")

print(pricing_details)

2022-10-14 11:48:09.435 | INFO     | __main__:<module>:5 - scraping: https://www.tripadvisor.com//Hotel_Review-g60763-d93585-Reviews-or480-Lowell_Hotel-New_York_City_New_York.html
2022-10-14 11:48:14.670 | INFO     | __main__:<module>:24 - found 0 prices from []


[]


### Post the payload to TripAdvisor and Scrape the prices

In [264]:
# Load scraped TripAdvisor hotels info dataset
ta_hotels_df = pd.read_pickle('./data/ta_hotels.pickle.zip')

hotel = 'Secret Bay'

url = ta_hotels_df.loc[ta_hotels_df.hotel == hotel, 'url'].values[0]
print(url)

page_data = ta_hotels_df.loc[ta_hotels_df.hotel == hotel, 'page'].values[0]
hotel_cache = extract_named_urql_cache(page_data["urqlCache"], '"locationDescription"')
hotel_info = hotel_cache["locations"][0]
info = parse_hotel_info(hotel_info)

hotelId = info['id']
print(hotelId)

checkInDate = "2023-05-07"
checkOutDate = "2023-05-11"

https://www.tripadvisor.com//Hotel_Review-g667237-d2104242-Reviews-or600-Secret_Bay-Portsmouth_Saint_John_Parish_Dominica.html
2104242


In [265]:
payload = json.dumps(
  [
    {
      "query": "eaea6d7eb0208f01fe0ca7d27238cbc1",
      "variables": {
        "request": {
          "hotelId": hotelId,
          "trackingEnabled": True,
          "requestCaller": "Hotel_Review",
          "impressionPlacement": "HR_MainCommerce",
          "pageLoadUid": "4c234841-fa05-48dd-9cab-13255d6ecb30",
          "sessionId": "FBD040A0F2094BB8BEAE17EDA8C00A45",
          "currencyCode": "USD",
          "requestNumber": 1,
          "shapeStrategy": "DEFAULT_DESKTOP_OFFER_SHAPE",
          "optimusEnabled": True,
          "sequenceId": 0,
          "travelInfo": {
            "checkInDate": checkInDate,
            "checkOutDate": checkOutDate,
            "usedDefaultDates": False,
            "rooms": 1,
            "adults": 1,
            "childAgesPerRoom": []
          },
          "allowOptimusDisplayPrice": False
        }
      }
    }
  ]
)

query_url = 'https://www.tripadvisor.com/data/graphql/ids'

headers = {
    # we need to generate a random request ID for this request to succeed
    "content-type": "application/json",
    "x-requested-by": "".join(random.choice(string.ascii_lowercase + string.digits) for i in range(64)),
}

secrets_scrapfly = hidden.scrapfly()
key = secrets_scrapfly['key']
with ScrapflyClient(key=key, max_concurrency=20) as session:
    """Scrape hotel pricing"""
    log.info(f"scraping: {url}")
    result = session.scrape(
        ScrapeConfig(
            url=query_url,
            country="US",
            headers=headers,
            body=payload,
            method="POST",
            asp=True,
        )
    )
    data = json.loads(result.content)

2022-10-14 16:08:44.929 | INFO     | __main__:<module>:45 - scraping: https://www.tripadvisor.com//Hotel_Review-g667237-d2104242-Reviews-or600-Secret_Bay-Portsmouth_Saint_John_Parish_Dominica.html


In [266]:
pricing_details = list()
price_page = data[0]['data']['HPS_getWebHROffers']
for offer_key in ['chevronOffers', 'textLinkOffers']:
    if any([source['status'] == 'AVAILABLE' for source in price_page[offer_key]]):
        for source in price_page[offer_key]:
            if source['status'] == 'AVAILABLE':
                pricing = dict()
                pricing['vendorName'] = source['data']['dataAtts']['vendorName']
                pricing['perNight'] = source['data']['dataAtts']['perNight']
                pricing_details.append(pricing)

log.info(f"found {len(pricing_details)} prices from {[pricing['vendorName'] for pricing in pricing_details]}")

print(pricing_details)

2022-10-14 16:08:49.442 | INFO     | __main__:<module>:12 - found 2 prices from ['Booking.com', 'eDreams']


[{'vendorName': 'Booking.com', 'perNight': 1578}, {'vendorName': 'eDreams', 'perNight': 1578}]


In [268]:
print(url)
print(hotel)

https://www.tripadvisor.com//Hotel_Review-g667237-d2104242-Reviews-or600-Secret_Bay-Portsmouth_Saint_John_Parish_Dominica.html
Secret Bay


### Insert the result to Elasticsearch

In [273]:
# Set up Elasticsearch connection
secrets = hidden.elastic()

es = Elasticsearch(
    f"https://{secrets['host']}:{secrets['port']}",
    ca_certs=secrets['ca'],
    basic_auth=(secrets['user'], secrets['pass']),
    verify_certs=False
)

# Generate a sha256 value from `url` for Elasticsearch doc id
m = hashlib.sha256()
m.update(url.encode())
pkey = m.hexdigest()

# Insert `pricing_details` to Elasticsearch
res = es.index(index=key, id=pkey, document={"group": "price", "price": pricing_details})
print(f"Added document as index 'price'")
print(res['result'])

Added document as index 'price'
created


### Insert the result to PostgreSQL and save to Pickle file

In [274]:
# Set up PostgreSQL connection
sql_string = hidden.psycopg2(hidden.postgres())
conn = psycopg2.connect(sql_string, connect_timeout=3)

# Insert result to PostgreSQL database
with conn.cursor() as cur:
    sql = """UPDATE hotels
               SET price = %s 
               WHERE hotel = %s;"""
    cur.execute(sql, (extras.Json(pricing_details), hotel))
conn.commit()
conn.close()

# Insert result to df
ta_hotels_df.at[ta_hotels_df[ta_hotels_df.hotel == hotel].index.values[0], 'price'] = pricing_details
ta_hotels_df.to_pickle('./data/ta_hotels.pickle.zip')

## Refresh Database Indices

### Recreate PostgreSQL Indices

In [ ]:
# Set up PostgreSQL connection
sql_string = hidden.psycopg2(hidden.postgres())
conn = psycopg2.connect(sql_string, connect_timeout=3)

# Create indices for `hotels` table
columns = ['hotel', 'url', 'info', 'price', 'reviews', 'page']
with conn.cursor() as cur:
    for column in columns:
        if column in ['hotel', 'url']:
            continue
        else:
            sql = f"DROP INDEX IF EXISTS {column}_gin;"
            cur.execute(sql)
            sql = f"CREATE INDEX {column}_gin ON hotels USING gin ({column});"
            cur.execute(sql)

            sql = f"DROP INDEX IF EXISTS {column}_gin_path_ops;"
            cur.execute(sql)
            sql = f"CREATE INDEX {column}_gin_path_ops ON hotels USING gin ({column} jsonb_path_ops);"
            cur.execute(sql)
conn.commit()

conn.close()

### Refresh Elasticsearch Indices

In [ ]:
# Set up Elasticsearch connection
secrets = hidden.elastic()

es = Elasticsearch(
    f"https://{secrets['host']}:{secrets['port']}",
    ca_certs=secrets['ca'],
    basic_auth=(secrets['user'], secrets['pass']),
    verify_certs=False
)

# Tell it to recompute the Elasticsearch indices - normally it would take up to 30 seconds
for column in columns:
    if column == 'url': continue
    res = es.indices.refresh(index=column)
    print(f"Index {column} refreshed")
    print(res)

## Record Library Dependency

In [2]:
%load_ext watermark
%watermark -u -i -d -v -iv -w -p loguru,scrapfly,elasticsearch

Last updated: 2022-10-09T08:46:23.211373+08:00

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.5.0

loguru       : 0.6.0
scrapfly     : 0.8.2
elasticsearch: 8.4.2

pandas  : 1.5.0
re      : 2.2.1
json    : 2.0.9
sys     : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:00:52) 
[Clang 13.0.1 ]
psycopg2: 2.9.3

Watermark: 2.3.1

